<a href="https://colab.research.google.com/github/24p11/recode-with-mistral-finetune/blob/main/tutorials/generate_fictives_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import re
from random import sample,randint
import hashlib
from mistralai import Mistral

#### Import ICD-10

In [55]:
PATH_ICD = "../referentials/"
df_icd = pd.read_csv(PATH_ICD +"cim_10_atih_2019.tsv", sep="\t",header=None,names=["code","aut_mco","pos","aut_ssr","lib_court","libelle"])
df_icd.code = df_icd.code.apply(lambda x: x.replace(" ",""))

#### Select some principal and associated diagnosis

Amoung the most frequent ICD-10 diagnisis use in French national DRG database we select a few.
We create a specific dataframe for DP dans DAS.

In [3]:
freq_cat_dp = "C50|H25|C34|C61|D12|C90|C18|D50|K29|I25|I50|I48|R10|C25|C67|H26|C20|C79|F10|C44|K57|K63|R07|K50"
freq_das = "I10|E1198|N185|G473|E43|C787|I482|C795|E559|B962|E1190|C780|E8718|I255|E8768|R410|R33"
df_dp = df_icd[df_icd.code.str.contains(freq_cat_dp) & (df_icd.aut_mco==0)]
df_das = df_icd[df_icd.code.str.contains(freq_das) & (df_icd.aut_mco==0)]

#### Functions to sample ICD-10 diagnosis in oder to build fictional scenario to give to  AI


In [36]:

def get_fictional_scenario(df_dp : pd.DataFrame , 
                  df_das: pd.DataFrame, 
                  n_das :int = None, 
                  max_das : int= 7, 
                  los : int = None, 
                  max_los : int = 10) :
    """
    Function to sample clinical scenario made of 1 principal diagnosis, n_das associated diagnosis and a length of staty
    
    Arguments
    -------------------
    df_dp : DataFrame. 2 colums at least ["libelle","code"]. ICD-10 pincipal diagnosis 
    df_das : DataFrame. 2 colums at least ["libelle","code"]. ICD-10 associated diagnosis
    n_das : int. number of associated diagnosis. Optionnal (the function can sample a number)
    max_das : int. maximal number of DAS if to sampled (n_das = None). Defaut 7.
    los : int. length of stay. Optional  (the function can sample a number)
    max_los :  int. Maximal length of stay if to sampled (n_das = None). Defaut 10.

    Related diagnosis are picked only if DP is a cancer (first letter of ICD-10 code = C).
    In that case, the function sample between 3 situations : 
    - chemotherapy  : DP = Z511, DR : sampled DP
    - radiotherapy  : DP = Z5101, DR : sampled DP
    - classical hopsitalisation :  DP = sampled DP, DR : None

    When n_das = 0 (given or sampled by the function) los between 0-2.

    Results
    ------------------
    df_diags :  DataFrame with 3 colmuns = concept_name, concept_code, condition_status_source_value. 
                Will be use to build the fictinal ICD-10 database. 
    los :       Int. Sampled length of stay
    text_diags: String. The ICD-10 diagnosis in the format 
                "
                Diagnostic principal : libelle (code)
                Diagnostic relié : libelle (code) ou aucun
                Diagnostic principal : libelle (code) ou aucun
                "
    """
    
    # Sample n_das if not given
    if n_das is None:
        n_das = randint(0,max_das)

    # Sample DP from df_dp
    prep = df_dp.sample(1).reset_index().loc[:,["libelle","code"]].rename(columns = {"code" : "concept_code","libelle" : "concept_name"})
    prep["condition_status_source_value"] = "DP"
    
    # if DP is cancer sample between chemotherapy, radiotherapy and classical hopsitalisation
    if prep.concept_code[0][0] =="C" :

        motive = sample(["Z511","Z5101","HC"],1)[0]
        if  motive=="Z511":
            prep["condition_status_source_value"] = "DR"
            prep = pd.concat([prep, 
                              pd.DataFrame(
                                  {'concept_name': 'Séance de chimiothérapie pour tumeur', 
                                   'concept_code': "Z511", 
                                   "condition_status_source_value": "DP"},
                                  index = [0])],
                             ignore_index=True)
            los = 0
        elif  motive=="Z5101":
            prep["condition_status_source_value"] = "DR"
            prep = pd.concat([prep, 
                              pd.DataFrame(
                                  {'concept_name': 'Séance de radiothérapie', 
                                   'concept_code': "Z5101", 
                                   "condition_status_source_value": "DP"},
                                  index = [0])],
                                  ignore_index=True)
            los = 0
        # if motive = HC then no DR
        else :
            prep = pd.concat([prep, 
                              pd.DataFrame(
                                  {'concept_name': np.nan, 
                                   'concept_code': np.nan, 
                                   "condition_status_source_value": "DR"},
                                  index = [0])],
                             ignore_index=True)
    # else no DR
    else :
        prep = pd.concat([prep, 
                              pd.DataFrame(
                                  {'concept_name': np.nan, 
                                   'concept_code': np.nan, 
                                   "condition_status_source_value": "DR"},
                                  index = [0])],
                         ignore_index=True)

    #Choose length of stay and associated diagnosis
    if n_das == 0:

        #los : between 0 and 2 if no associated diagnosis
        if los is None :  # do not modify if already fixed during previous steps
            los = randint(0,2)
        
        #No DAS
        df_diags_prep = pd.concat([prep, 
                              pd.DataFrame(
                                  {'concept_name': np.nan, 
                                   'concept_code': np.nan, 
                                   "condition_status_source_value": "DAS"},
                                  index = [0])],
                                  ignore_index=True)

        
    else : 
        
        #los between 3 and 10 days
        if los is None : # do not modify if already fixed during previous steps
            los = randint(3,max_los)
        
        #pick some DAS from df_das
        prep_ = df_das.sample(n_das).reset_index().loc[:,["libelle","code"]].rename(columns = {"code" : "concept_code","libelle" : "concept_name"})
        prep_["condition_status_source_value"] = "DAS"
        df_diags_prep = pd.concat([prep,prep_])

    # Prepare the final DataFrame of ICD diagnosis (only take lines whith an effective diagnosis)
    df_diags = df_diags_prep[df_diags_prep.concept_code.notna()]

    # Concatenate ICD-10 label and code.
    df_diags_prep["diag"] = np.where(df_diags_prep.concept_name.notna(), df_diags_prep.concept_name + ' (' + df_diags_prep.concept_code.replace(" ","") +')' , 'Aucun' )
    df_diags_prep = df_diags_prep.groupby("condition_status_source_value")['diag'].apply(lambda x: ', '.join(x)).reset_index()

    text_diags = " Diagnostics CM-10 :\n- Diagnostic principal : " +  df_diags_prep.iloc[1,1] +".\n- Diagnostics relié : "+ df_diags_prep.iloc[2,1] +".\n" +"- Diagnostics associés :" +  df_diags_prep.iloc[0,1]
    return df_diags,los,text_diags

In [38]:
# test the function 
df_diags_prep,los, text_diags = get_fictional_scenario(df_dp ,df_das)
print( text_diags)
print("length of stay : " +  str(los))

 Diagnostics CM-10 :
- Diagnostic principal : Cataracte infantile, juvénile et présénile (H260).
- Diagnostics relié : Aucun.
- Diagnostics associés :Aucun
length of stay : 1


In [39]:
# test for cancer patient
df_diags_prep,los, text_diags = get_fictional_scenario(df_dp[df_dp.code.str.contains('C')] ,df_das)
print( text_diags)
print("length of stay : " +  str(los))

 Diagnostics CM-10 :
- Diagnostic principal : Séance de radiothérapie (Z5101).
- Diagnostics relié : Tumeur maligne du pancréas, autre et non précisée (C259+8).
- Diagnostics associés :Malnutrition protéino-énergétique grave, sans précision (E43), Désorientation, sans précision (R410), Diabète sucré de type 2 non insulinotraité ou sans précision, sans complication (E1198), Tumeur maligne secondaire des os et de la moelle osseuse (C795), Fibrillation auriculaire chronique [permanente] (I482), Hypokaliémies, autres et sans précision (E8768)
length of stay : 0


In [21]:
def get_visit_occurence_id():
    num = randint(1,1000000000000)
    m = hashlib.sha1(str(num).encode())
    return m.hexdigest()

In [7]:
get_visit_occurence_id()

'd77aaec8ffc57012439d5cc84a8642a1a0d30f04'

### Use Mistral API to wite fictional clinical case

Initialisation of the API function call

In [ ]:
api_key = os.getenv('MISTRAL_API_KEY')
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

In [9]:
# Test with a simple example.
chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)
print(chat_response.choices[0].message.content)

Determining the "best" French cheese can be quite subjective, as it often depends on personal taste. France is renowned for its wide variety of cheeses, with over 400 different types. Here are a few that are often highly regarded:

1. **Camembert de Normandie**: A soft, creamy, and slightly salty cheese from the Normandy region. It's one of the most famous French cheeses worldwide.

2. **Brie de Meaux**: Often referred to as the "King of Cheeses," this soft cheese is known for its rich, creamy interior and slightly crumbly crust.

3. **Roquefort**: A classic blue cheese made from sheep's milk, known for its strong flavor and creamy texture.

4. **Comté**: A hard cheese made from unpasteurized cow's milk, with a complex, nutty flavor. It's often used in both cooking and as a standalone cheese.

5. **Chèvre**: This is a general term for goat cheese, which can range from soft and creamy to hard and crumbly. Fresh chèvre is often coated in herbs or ash.

6. **Reblochon**: A soft washed-rin

#### Build specific prompt for the use case

In [10]:
prompt_system_intructions = """
Contexte :
Vous êtes un médecin expérimenté rédigeant un compte rendu d'hospitalisation pour un patient fictif à partir d'un scénario qui comprend les éléments suivants :
codes de la classification internationnale des maladies version 10 (CIM-10) et durée du séjour.
Le compte rendu doit être littéraire, avec un vocabulaire non formel, et doit inclure des détails cliniques précis pour compenser le côté "macroscopique" 
des diagnostics médicaux qui sont fourni dans le format des codes de la CIM-10.

Instructions :
Utilisez les codes CIM-10 fournis pour guider la création du compte rendu, mais éloignez-vous du vocabulaire formel de la CIM-10.
Vous n'utiliserez que les diagnostics compatibles avec les codes CIM-10 fournis, mais vous essayerez de donner plus de détails pour rendre le compte rendu réaliste et détaillé,
Incluez des détails sur les antécédents médicaux, les examens cliniques, les résultats des examens complémentaires, le traitement, en donnant le nom des molécules et les posologies lorsqu'il s'agit de traitement médicamenteux, et l'évolution du patient.
Ajoutez des résultats biologiques précis avec des valeurs normales et des précisions sur des pathologies associées fictives.
Le compte rendu doit être assez long, et vous veillerez à utiliser un ton littéraire et un vocabulaire non formel pour rendre le compte rendu plus engageant.
"""
prompt_system_crh_exemple = """
Exemple demande utilisateur :

Diagnostic principal : [Code CIM-10]
Diagnostic relié : [Code CIM-10]
Diagnostics associés : [Liste de codes CIM-10]

Exemple réponse attentue :

Compte Rendu d'Hospitalisation

Identité du patient :

Nom : [Nom du patient]
Prénom : [Prénom du patient]
Âge : [Âge du patient]
Sexe : [Sexe du patient]
Mode d'entrée : [Mode d'entrée]

Durée de l'hospitalisation : [Durée de l'hospitalisation données par l'utilisateur]

Service : [Service]

Motif d'admission :
[Détails sur le motif d'admission, en utilisant un ton littéraire et des détails cliniques précis]

Antécédents médicaux :
[Détails sur les antécédents médicaux du patient, en utilisant un ton littéraire et des détails cliniques précis]

Examen clinique à l'admission :
[Détails sur l'examen clinique à l'admission, en utilisant un ton littéraire et des détails cliniques précis]

Résultats biologiques :
[Détails sur les résultats biologiques précis avec des valeurs normales et des précisions sur des pathologies associées fictives]

Résultats des autres examens complémentaires :
[Détails sur les résultats des examens complémentaires, en utilisant un ton littéraire et des détails cliniques précis]

Traitement et évolution :
[Détails sur le traitement et l'évolution du patient, en utilisant un ton littéraire et des détails cliniques précis]

Conclusion et recommandations :
[Détails sur la conclusion et les recommandations, en utilisant un ton littéraire et des détails cliniques précis]

Date d'admission : [Date d'admission]
Date de sortie : [Date de sortie]

Signé :
Dr. [Nom du médecin]
"""


prompt_exemple_specifique ="""
Peux tu produire un CRH détaillé à partir du scénario suivant :

Diagnostic principal : Infarctus aigu du myocarde de la paroi antérieure (I21.0)
Diagnostic relié : Aucun.
Diagnostics associés :  Athérosclérose cardiaque (I25.1), Diabète de type 2 sans complication (E11.9), Hypertension essentielle (primitive)  (I10).

Durée de l'hopsitalisation = 7 jours.

Résultat attendu :

Compte Rendu d'Hospitalisation

Identité du patient :

Nom : Martin
Prénom : Pierre
Âge : 58 ans
Sexe : Masculin
Mode d'entrée : Urgences

Durée de l'hospitalisation : 7 jours

Service : Cardiologie

Motif d'admission :
Pierre Martin, un homme de 58 ans, a été admis aux urgences en raison de douleurs thoraciques intenses et de difficultés respiratoires. Il avait récemment subi un infarctus aigu du myocarde de la paroi antérieure, et des complications étaient apparues, nécessitant une attention immédiate.

Antécédents médicaux :
Pierre a un passé médical complexe, marqué par une athérosclérose cardiaque, un diabète de type 2 et une hypertension essentielle. Ces conditions ont compliqué son état de santé et nécessité une surveillance étroite.

Examen clinique à l'admission :
À son arrivée, Pierre présentait des signes vitaux instables, avec une pression artérielle élevée et une fréquence cardiaque rapide. L'examen physique a révélé des douleurs thoraciques intenses et des signes de détresse respiratoire.

Résultats des examens complémentaires :
Les analyses de laboratoire ont montré des niveaux élevés de troponine, confirmant un infarctus aigu du myocarde. L'électrocardiogramme a révélé des anomalies significatives, et l'échocardiographie a montré une réduction de la fonction ventriculaire gauche.

Résultats biologiques :

Troponine : 0.5 ng/mL (valeur normale < 0.04 ng/mL)
Glycémie : 180 mg/dL (valeur normale < 140 mg/dL)
Hémoglobine A1c : 7.5% (valeur normale < 5.7%)
Créatinine : 1.2 mg/dL (valeur normale < 1.3 mg/dL)
Cholestérol total : 250 mg/dL (valeur normale < 200 mg/dL)
LDL : 160 mg/dL (valeur normale < 100 mg/dL)
HDL : 40 mg/dL (valeur normale > 40 mg/dL)
Triglycérides : 200 mg/dL (valeur normale < 150 mg/dL)
Pathologies associées fictives :
Pierre présente également une insuffisance rénale chronique légère et une hypercholestérolémie, compliquant encore son état de santé.

Traitement et évolution :
Pierre a reçu un traitement médical intensif, incluant des anticoagulants, des bêta-bloquants et des inhibiteurs de l'enzyme de conversion. Une angioplastie coronarienne a été réalisée pour rétablir le flux sanguin dans les artères coronaires. Au fil des jours, son état général s'est progressivement amélioré, grâce à la vigilance et aux soins attentifs de l'équipe médicale.

Conclusion et recommandations :
Après une hospitalisation de 7 jours, Pierre a pu quitter l'hôpital avec un diagnostic final d'infarctus aigu du myocarde de la paroi antérieure. Pour assurer un suivi optimal, il est recommandé de surveiller régulièrement la fonction cardiaque, de suivre de près l'évolution de l'hypertension et du diabète, et de continuer le traitement médical prescrit. Des visites de suivi régulières sont également nécessaires pour s'assurer qu'il n'y a pas de récidive de l'infarctus.



"""


Test with an exemple

In [35]:
chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "system",
            "content": prompt_system_intructions ,
        },
        {
            "role": "system",
            "content": prompt_exemple_specifique ,
        },    
        {
            "role": "user",
            "content": """ Peux tu produire un CRH détaillé à partir des codes CIM-10 suivants :
                            Diagnostics CM-10 :
                            - Diagnostic principal : Diverticulose de l'intestin, (siège non précisé, sans perforation ni abcès) (K579).
                            - Diagnostics relié : Aucun.
                            - Diagnostics associés :Fibrillation auriculaire chronique [permanente] (I482), Désorientation, sans précision (R410), Carence en vitamine D, sans précision (E559)
                            Durée de l'hospitalisation : 9
                            """,
        },
    ]
)
print(chat_response.choices[0].message.content)

**Compte Rendu d'Hospitalisation**

**Identité du patient :**

Nom : Dupont
Prénom : Jean
Âge : 65 ans
Sexe : Masculin
Mode d'entrée : Urgences

**Durée de l'hospitalisation :** 9 jours

**Service :** Gastroentérologie

**Motif d'admission :**
Jean Dupont, un homme de 65 ans, a été admis aux urgences en raison de douleurs abdominales sévères et de symptômes digestifs inquiétants. Il a récemment été diagnostiqué avec une diverticulose de l'intestin, sans perforation ni abcès, mais ses symptômes se sont aggravés, nécessitant une prise en charge hospitalière immédiate.

**Antécédents médicaux :**
Jean a un passé médical chargé. Il souffre de fibrillation auriculaire chronique, ce qui complique la gestion de sa condition cardiaque. De plus, il présente des épisodes récurrents de désorientation, sans cause précise identifiée, et une carence en vitamine D, qui a été détectée lors de ses derniers bilans sanguins. Ces conditions ont toutes contribué à la complexité de son état de santé actuel.

#### Build the fictionals clinical case
Result 2 files
- clinical_notes.csv : visit_occurence_id, text
- codes.csv : visit_occurence_id,condition_status_source_value,concept_code,concept_name

In [49]:
# Finals dataframes
df_texts = pd.DataFrame(columns=['visit_occurence_id', 'text',' scenario'])
cols_diags = ['visit_occurence_id', 'condition_status_source_value','concept_code','concept_name']
df_diags = pd.DataFrame(columns=cols_diags)

for i in range(0,200):

    diags,los,text_diags = get_fictional_scenario(df_dp ,df_das)

    id = get_visit_occurence_id()

    diags["visit_occurence_id"] = id


    df_diags = df_diags.append(diags[cols_diags])

    user_content = text_diags + "\nDurée d'hospitalisaiton  = " + str(los) + " jours."

    chat_response = client.chat.complete(
        model= model,
        messages = [
            {
                "role": "system",
                "content": prompt_system_intructions ,
            },
            {
                "role": "system",
                "content": prompt_exemple_specifique ,
            },    
            {
                "role": "user",
                "content": user_content,
            },
        ]
    )

    df_texts = df_texts.append({'visit_occurence_id': id, 'text': chat_response.choices[0].message.content, 'scenario' : user_content }, ignore_index=True)

In [50]:
df_diags

,visit_occurence_id,condition_status_source_value,concept_code,concept_name
0,5e6f73d5427e6e6ca6be572947d25d792d760188,DP,R102,Douleur pelvienne et périnéale
0,5e6f73d5427e6e6ca6be572947d25d792d760188,DAS,E8768,"Hypokaliémies, autres et sans précision"
1,5e6f73d5427e6e6ca6be572947d25d792d760188,DAS,E1198,Diabète sucré de type 2 non insulinotraité ou ...
2,5e6f73d5427e6e6ca6be572947d25d792d760188,DAS,I10,Hypertension essentielle (primitive)
3,5e6f73d5427e6e6ca6be572947d25d792d760188,DAS,C795,Tumeur maligne secondaire des os et de la moel...
...,...,...,...,...
2,cd372bd9dfd0e15f1281bfb1496ff3dd3346e4dd,DAS,E8718,"Hypoosmolarités et hyponatrémies, autres et sa..."
3,cd372bd9dfd0e15f1281bfb1496ff3dd3346e4dd,DAS,I482,Fibrillation auriculaire chronique [permanente]
4,cd372bd9dfd0e15f1281bfb1496ff3dd3346e4dd,DAS,N185,"Maladie rénale chronique, stade 5"
5,cd372bd9dfd0e15f1281bfb1496ff3dd3346e4dd,DAS,E43,"Malnutrition protéino-énergétique grave, sans ..."


In [51]:
print(df_texts.loc[0,'scenario'])

 Diagnostics CM-10 :
- Diagnostic principal : Douleur pelvienne et périnéale (R102).
- Diagnostics relié : Aucun.
- Diagnostics associés :Hypokaliémies, autres et sans précision (E8768), Diabète sucré de type 2 non insulinotraité ou sans précision, sans complication (E1198), Hypertension essentielle (primitive) (I10), Tumeur maligne secondaire des os et de la moelle osseuse (C795)
Durée d'hospitalisaiton  = 4 jours.


In [52]:
print(df_texts.loc[0,'text'])

**Compte Rendu d'Hospitalisation**

**Identité du patient :**

Nom : Dupont
Prénom : Jean
Âge : 62 ans
Sexe : Masculin
Mode d'entrée : Urgences

**Durée de l'hospitalisation :** 4 jours

**Service :** Médecine Interne

**Motif d'admission :**
Jean Dupont, un homme de 62 ans, a été admis aux urgences en raison de douleurs pelviennes et périnéales intenses qui l'ont réveillé en pleine nuit. Ces douleurs étaient accompagnées d'une sensation de malaise général et d'une faiblesse musculaire marquée.

**Antécédents médicaux :**
Jean a un passé médical chargé. Il souffre de diabète de type 2, traité par des antidiabétiques oraux, et d'une hypertension essentielle qui nécessite une surveillance régulière. Récemment, il a été diagnostiqué avec une tumeur maligne secondaire des os et de la moelle osseuse, une nouvelle qui a bouleversé sa vie et celle de ses proches. De plus, il présente des épisodes récurrents d'hypokaliémie, ce qui complique encore son état de santé.

**Examen clinique à l'admi

In [57]:
df_texts[["visit_occurence_id","text"]].to_csv("../sample_data/text.csv",index=False)

In [56]:
df_diags.to_csv("../sample_data/codes.csv",index=False)